In [1]:
from py2neo import Graph, Node, Relationship, Subgraph
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import re,string

In [39]:
def SimilarByCol(col, data, topn=10, exclude=[], duplicate=0):
    mask_x= data.index.isin(col)
    mask_y= ~data.index.isin(col + exclude)
    sim = pd.DataFrame(cosine_similarity(data[mask_y].values,data[mask_x].values),
                       index=data[mask_y].index, columns=data[mask_x].index)
    # sim = data.loc[~data.index.isin(col + exclude), col]
    if duplicate == 0:
        return [(co, list(sim[co].sort_values(ascending=False)[:topn].items())) for co in col]  # 不对相同节点关系去重,保存排序顺序

    similar_next = {}  # 数据格式:{co:[(next,val)*topn]}
    if duplicate == 1:  # 由上一节点顺序优先选择
        for co in col:
            y = list(sim[co].sort_values(ascending=False)[:topn * len(col)].items())
            y2 = [_[0] for x in similar_next.values() for _ in x]
            similar_next[co] = [y3 for y3 in y if y3[0] not in y2][:3]

    if duplicate == 2:  # 各自排序后,由下一节点各自选最近的上节点,最后由上节点各自匹配重复关系
        similar_with = {}  # {y:(x,v)}
        for co in col:
            y = list(sim[co].sort_values(ascending=False)[:topn * len(col)].items())  # 最多重复目标数量:(n*m)*m
            for z in y:
                if z[0] not in similar_with or z[1] > similar_with[z[0]][1]:  # 对关系去重选最大的,反向选择
                    similar_with[z[0]] = (co, z[1])

        for y, x in similar_with.items():
            similar_next[x[0]] = sorted(similar_next.get(x[0], []) + [(y, x[1])], key=lambda z: z[1], reverse=True)[
                                 :topn]

    if duplicate == 3:
        # [(row.Index, col, sim.at[row.Index, col]) for row in sim.itertuples() for col in sim.columns]
        # similar_sort=sorted([(i, j, sim.iloc[i, j]) for i in range(sim.shape[0]) for j in range(sim.shape[1])],
        #         key=lambda x:x[2],reverse=True)#所有数据值排序(row,col,value)
        similar_sort = sorted(enumerate(sim.values.reshape(-1)), key=lambda x: x[1], reverse=True)  # 所有数据值排序(i,value)
        pos = []  # 储存行列位置数据,统计数量与去重
        mx = len(col)
        for i, v in similar_sort:
            y = [p[0] for p in pos]
            if len(y) >= topn * mx:
                break
            ix = i % mx
            iy = i // mx
            z = [p[1] for p in pos if p[1] == ix]
            if len(z) < topn and iy not in y:
                pos.append((iy, ix))  # (x[0],x[1])
                similar_next[sim.columns[ix]] = similar_next.get(sim.columns[ix], []) + [(sim.index[iy], v)]

    return [(co, similar_next[co]) for co in col if co in similar_next]  # 恢复顺序 list(similar_next.items())
    # 返回数据格式:[(co:[(next,value)*topn])*len(col)],皆有排序

In [3]:
co_ids_vec=pd.read_parquet('data/patent_co_vec_先进制造_医疗健康.parquet')

In [42]:
co_ids_vec.loc['万向一二三股份公司'].values

array([-0.68488471,  0.58515707, -0.06731017,  0.24676351, -0.36817273,
        0.33369659,  1.00222055,  0.67487162, -0.72877585,  0.18888464,
        0.05701108, -0.02932522, -0.76778367, -0.3862036 , -0.11951465,
       -0.24621082,  0.29444565,  0.2053027 , -0.36896775, -0.16868726,
       -0.20525941,  0.40116362,  0.42188984, -0.17877991, -0.1865988 ,
        0.63828   , -0.45213638,  0.07975333,  0.20553199, -0.2282062 ,
       -0.14579283,  0.73533006, -0.02172106, -0.32476774,  0.35330559,
       -0.3668453 , -0.31186834, -0.48242104,  0.14614808,  0.21587111,
        0.13650062, -0.35354748, -0.13264049,  0.67643863, -0.11523288,
       -0.08948488, -0.40477627, -0.07739272,  0.07516872, -0.33399974,
        0.73978404, -0.49505331,  0.76774891, -0.2880875 ,  0.13322648,
       -0.50915536,  0.05690048, -0.36805762,  0.7808923 ,  0.47191681,
        0.1944226 , -0.40105207, -0.51240712, -0.80118255,  0.47411647,
        0.14715215, -0.54593963, -0.01327396, -0.35918726, -0.32

In [33]:
SimilarByCol(['北京鼎材科技有限公司'], co_ids_vec, topn=3, exclude=list({'北京鼎材科技有限公司':0}),
                                              duplicate=0)

Co              北京鼎材科技有限公司
Co                        
Dexta Robotics    0.311426
TCL华星光电技术有限公司     0.620696
TCL科技集团股份有限公司     0.777362
一品红药业股份有限公司       0.738592
一心堂药业集团股份有限公司     0.180454
...                    ...
鼎桥通信技术有限公司        0.330140
龙微科技无锡有限公司        0.613320
龙晖药业有限公司          0.476626
龙腾半导体股份有限公司       0.842942
龙芯中科技术股份有限公司      0.271378

[10979 rows x 1 columns] Index(['Dexta Robotics', 'TCL华星光电技术有限公司', 'TCL科技集团股份有限公司', '一品红药业股份有限公司',
       '一心堂药业集团股份有限公司', '一道新能源科技股份有限公司', '七星电气股份有限公司', '七腾机器人有限公司',
       '万华新材料有限公司', '万向一二三股份公司',
       ...
       '黑龙江天晴干细胞股份有限公司', '黑龙江威凯洱生物技术有限公司', '黑龙江林海华安新材料股份有限公司',
       '黑龙江珍宝岛药业股份有限公司', '黑龙江省发现者机器人股份有限公司', '鼎桥通信技术有限公司', '龙微科技无锡有限公司',
       '龙晖药业有限公司', '龙腾半导体股份有限公司', '龙芯中科技术股份有限公司'],
      dtype='object', name='Co', length=10979) Index(['北京鼎材科技有限公司'], dtype='object', name='Co')


[('北京鼎材科技有限公司',
  [('吉林奥来德光电材料股份有限公司', 0.9990528910524518),
   ('长春海谱润斯科技股份有限公司', 0.9981385542976898),
   ('北京绿人科技有限责任公司', 0.9980779351494903)])]

In [37]:
def CreateNode(word, depth, graph, node_name='Word', **kwargs):
    nodes = graph.nodes.match(node_name)
    node = nodes.where(name=word).first()  # Node("Word", name=word,id=mid)
    if not node:
        node = Node(node_name, name=word, depth=depth, nid=len(nodes), **kwargs)
        graph.create(node)
    return node

def CreateCoSimilarityRelationship(co, data, graph=None, width=3, max_depth=3, wid=0, duplicate=0):  # 层级扩散
    words_depth = {co: 0}
    similar_words = [co]
    depth = 0
    while depth <= max_depth:
         try:
            similar_words_next = SimilarByCol(similar_words, data, topn=width, exclude=list(words_depth),
                                              duplicate=duplicate)  # 分支会有合并,下层关系去重
            similar_words = list(set(y[0] for x in similar_words_next for y in x[1]))  # 迭代词组
            print("Depth:", depth, "Similar:", [x[0] for x in similar_words_next], "->", similar_words_next)
            depth += 1
            words_depth.update({w: depth for w in similar_words if w not in words_depth})

            if not graph:
                continue

            node_name = f"Co_{wid}"
            for x in similar_words_next:
                word_node = CreateNode(x[0], words_depth[x[0]], graph, node_name,
                                       similar=float(data.loc[co, x[0]]))  # w=x[0]
                relationships = [Relationship(word_node, "SIMILAR_TO",
                                              CreateNode(y[0], words_depth[y[0]], graph, node_name,
                                                         similar=float(data.loc[co, y[0]])), rank=i,
                                              similarity=float(y[1]))
                                 for i, y in enumerate(x[1])]
                print(relationships)
                graph.create(Subgraph(relationships=relationships))  # 创建关系

         except KeyError:
            print(f"Data '{co, similar_words}' not in vocabulary.")
            break

    return words_depth

In [40]:
CreateCoSimilarityRelationship('北京鼎材科技有限公司', co_ids_vec)

Depth: 0 Similar: ['北京鼎材科技有限公司'] -> [('北京鼎材科技有限公司', [('吉林奥来德光电材料股份有限公司', 0.9990528910524518), ('长春海谱润斯科技股份有限公司', 0.9981385542976898), ('北京绿人科技有限责任公司', 0.9980779351494903)])]
Depth: 1 Similar: ['北京绿人科技有限责任公司', '吉林奥来德光电材料股份有限公司', '长春海谱润斯科技股份有限公司'] -> [('北京绿人科技有限责任公司', [('江苏三月科技股份有限公司', 0.9987378495033277), ('北京云基科技股份有限公司', 0.9961204241066461), ('浙江华显光电科技有限公司', 0.9960389749613339)]), ('吉林奥来德光电材料股份有限公司', [('江苏三月科技股份有限公司', 0.9988027569779777), ('北京云基科技股份有限公司', 0.9974503588605215), ('浙江华显光电科技有限公司', 0.9972747993961116)]), ('长春海谱润斯科技股份有限公司', [('江苏三月科技股份有限公司', 0.9986203289653169), ('北京云基科技股份有限公司', 0.9967049010056677), ('浙江华显光电科技有限公司', 0.9963780013149999)])]
Depth: 2 Similar: ['浙江华显光电科技有限公司', '北京云基科技股份有限公司', '江苏三月科技股份有限公司'] -> [('浙江华显光电科技有限公司', [('江西冠能光电材料有限公司', 0.9956820462067055), ('上海钥熠电子科技有限公司', 0.9955295181503053), ('陕西莱特光电材料股份有限公司', 0.9948819897087462)]), ('北京云基科技股份有限公司', [('江西冠能光电材料有限公司', 0.9947581348069168), ('上海钥熠电子科技有限公司', 0.9929743470109249), ('陕西莱特光电材料股份有限公司', 0.9926311188069998)]), 

{'北京鼎材科技有限公司': 0,
 '北京绿人科技有限责任公司': 1,
 '吉林奥来德光电材料股份有限公司': 1,
 '长春海谱润斯科技股份有限公司': 1,
 '浙江华显光电科技有限公司': 2,
 '北京云基科技股份有限公司': 2,
 '江苏三月科技股份有限公司': 2,
 '江西冠能光电材料有限公司': 3,
 '上海钥熠电子科技有限公司': 3,
 '陕西莱特光电材料股份有限公司': 3,
 '宁波卢米蓝新材料有限公司': 4,
 '武汉长弢新材料有限公司': 4,
 '广州追光科技有限公司': 4,
 '南京高光半导体材料有限公司': 4,
 '北京夏禾科技有限公司': 4}

In [49]:
port_result_ew = pd.read_csv('data\initial_data_0428.csv',index_col=0)
port_result_ew_columns = ['excess_return', 'ff3', 'ff5', 'capm', 'carhrt', 'lsy4']
df_new = pd.DataFrame()
for trans_i in port_result_ew_columns:
    trans_i_df = port_result_ew[[trans_i,'portfolio_name']].explode(trans_i).reset_index()
    df_new= pd.merge(df_new,trans_i_df,left_index=True,right_index=True,how='outer')#MergeError: Passing 'suffixes' which cause duplicate columns {'portfolio_name_x'} is not allowed.
    print(df_new)
    if 'index_x' in df_new.columns:
        del df_new['index_x']

df_new=df_new.loc[:,~df_new.columns.duplicated()]
port_result_ew_columns.extend(['portfolio_name_x'])
port_result_ew_df = df_new[port_result_ew_columns].rename(columns={'portfolio_name_x':'portfolio_name'})
port_result_ew_df = port_result_ew_df.set_index(['portfolio_name','excess_return', 'ff3', 'ff5', 'capm', 'carhrt', 'lsy4'])
port_result_ew_df.to_excel('final_data')

    index            excess_return portfolio_name
0       0  ['-1.751**', '(-2.27)']             P1
1       1    ['-0.456', '(-0.54)']             P2
2       2      ['0.263', '(0.34)']             P3
3       3      ['0.703', '(0.85)']             P4
4       4      ['1.314', '(1.57)']             P5
5       5     ['1.467*', '(1.67)']             P6
6       6    ['2.111**', '(2.51)']             P7
7       7    ['2.74***', '(3.12)']             P8
8       8   ['3.771***', '(4.29)']             P9
9       9   ['5.295***', '(5.77)']            P10
10     10  ['7.047***', '(13.31)']            h_l
    index_x            excess_return portfolio_name_x  index_y  \
0         0  ['-1.751**', '(-2.27)']               P1        0   
1         1    ['-0.456', '(-0.54)']               P2        1   
2         2      ['0.263', '(0.34)']               P3        2   
3         3      ['0.703', '(0.85)']               P4        3   
4         4      ['1.314', '(1.57)']               P5        4   
5   

MergeError: Passing 'suffixes' which cause duplicate columns {'portfolio_name_x'} is not allowed.

In [56]:
port_result_ew.excess_return.values

array(["['-1.751**', '(-2.27)']", "['-0.456', '(-0.54)']",
       "['0.263', '(0.34)']", "['0.703', '(0.85)']",
       "['1.314', '(1.57)']", "['1.467*', '(1.67)']",
       "['2.111**', '(2.51)']", "['2.74***', '(3.12)']",
       "['3.771***', '(4.29)']", "['5.295***', '(5.77)']",
       "['7.047***', '(13.31)']"], dtype=object)

In [59]:
import ast
port_result_ew.excess_return.apply(lambda x: ast.literal_eval(x))

0     [-1.751**, (-2.27)]
1       [-0.456, (-0.54)]
2         [0.263, (0.34)]
3         [0.703, (0.85)]
4         [1.314, (1.57)]
5        [1.467*, (1.67)]
6       [2.111**, (2.51)]
7       [2.74***, (3.12)]
8      [3.771***, (4.29)]
9      [5.295***, (5.77)]
10    [7.047***, (13.31)]
Name: excess_return, dtype: object

In [148]:
import ast
port_result_ew = pd.read_csv('data\initial_data_0428.csv',index_col=0)
port_result_ew.set_index('portfolio_name',inplace=True)
df_new =pd.concat([port_result_ew[trans_i].apply(lambda x: ast.literal_eval(x)).explode()  
                   for trans_i in port_result_ew.columns],axis=1).reset_index()
df_new =df_new.groupby(['portfolio_name',df_new.index]).agg(lambda x:x).sort_index(level=1)


In [156]:
df_new

excess_return        ff3        ff5       capm     carhrt  \
portfolio_name                                                                
P1             0       -1.751**  -3.087***  -3.061***  -2.659***  -3.093***   
               1        (-2.27)   (-15.34)   (-14.02)   (-10.73)   (-15.09)   
P2             2         -0.456  -1.882***  -1.895***  -1.382***  -1.873***   
               3        (-0.54)   (-10.16)     (-9.3)    (-4.84)    (-10.3)   
P3             4          0.263  -1.085***  -1.038***   -0.635**  -1.063***   
               5         (0.34)    (-7.75)    (-6.71)    (-2.53)    (-7.57)   
P4             6          0.703  -0.707***  -0.635***     -0.211  -0.692***   
               7         (0.85)    (-4.54)     (-4.0)    (-0.68)    (-4.43)   
P5             8          1.314     -0.093     -0.097      0.395     -0.063   
               9         (1.57)    (-0.52)     (-0.6)     (1.24)    (-0.36)   
P6             10        1.467*      0.028      0.018     0.523*      0.045   
               11        (1.67)     (0.19)     (0.11)     (1.67)      (0.3)   
P7             12       2.111**   0.757***   0.727***   1.248***   0.773***   
               13        (2.51)     (4.88)     (4.93)     (3.69)     (5.01)   
P8             14       2.74***   1.251***   1.294***   1.786***   1.255***   
               15        (3.12)     (7.93)     (7.41)     (5.24)     (7.97)   
P9             16      3.771***   2.266***   2.268***   2.804***   2.244***   
               17        (4.29)    (12.72)    (13.07)     (7.54)    (12.28)   
P10            18      5.295***   3.855***   3.969***   4.331***   3.865***   
               19        (5.77)    (15.55)    (15.48)    (12.18)    (15.61)   
h_l            20      7.047***   6.942***   7.031***   6.989***   6.959***   
               21       (13.31)    (17.08)    (16.24)    (17.07)    (17.03)   

                        lsy4  
portfolio_name                
P1             0   -3.137***  
               1    (-14.45)  
P2             2   -2.047***  
               3    (-10.99)  
P3             4   -1.197***  
               5     (-7.61)  
P4             6   -0.838***  
               7     (-4.97)  
P5             8      -0.173  
               9     (-1.01)  
P6             10     -0.059  
               11    (-0.32)  
P7             12   0.645***  
               13     (4.43)  
P8             14    1.14***  
               15     (5.87)  
P9             16   2.092***  
               17    (11.51)  
P10            18   3.867***  
               19     (14.7)  
h_l            20   7.004***  
               21    (16.49)

In [138]:
df_new['index']=df_new.index.get_level_values(1)

In [144]:
df_new.reset_index()

,portfolio_name,level_1,excess_return,ff3,ff5,capm,carhrt,lsy4,index
0,P1,0,-1.751**,-3.087***,-3.061***,-2.659***,-3.093***,-3.137***,0
1,P1,1,(-2.27),(-15.34),(-14.02),(-10.73),(-15.09),(-14.45),1
2,P10,18,5.295***,3.855***,3.969***,4.331***,3.865***,3.867***,18
3,P10,19,(5.77),(15.55),(15.48),(12.18),(15.61),(14.7),19
4,P2,2,-0.456,-1.882***,-1.895***,-1.382***,-1.873***,-2.047***,2
5,P2,3,(-0.54),(-10.16),(-9.3),(-4.84),(-10.3),(-10.99),3
6,P3,4,0.263,-1.085***,-1.038***,-0.635**,-1.063***,-1.197***,4
7,P3,5,(0.34),(-7.75),(-6.71),(-2.53),(-7.57),(-7.61),5
8,P4,6,0.703,-0.707***,-0.635***,-0.211,-0.692***,-0.838***,6
9,P4,7,(0.85),(-4.54),(-4.0),(-0.68),(-4.43),(-4.97),7


In [ ]:
df_new['portfolio_name'], df_new['index'] = zip(*df_new.index)
df_new#.to_excel('data\\final_data.xlsx')

In [128]:
df_new.set_index('index')

,excess_return,ff3,ff5,capm,carhrt,lsy4,portfolio_name
index,,,,,,,
0,-1.751**,-3.087***,-3.061***,-2.659***,-3.093***,-3.137***,P1
1,(-2.27),(-15.34),(-14.02),(-10.73),(-15.09),(-14.45),P1
18,5.295***,3.855***,3.969***,4.331***,3.865***,3.867***,P10
19,(5.77),(15.55),(15.48),(12.18),(15.61),(14.7),P10
2,-0.456,-1.882***,-1.895***,-1.382***,-1.873***,-2.047***,P2
3,(-0.54),(-10.16),(-9.3),(-4.84),(-10.3),(-10.99),P2
4,0.263,-1.085***,-1.038***,-0.635**,-1.063***,-1.197***,P3
5,(0.34),(-7.75),(-6.71),(-2.53),(-7.57),(-7.61),P3
6,0.703,-0.707***,-0.635***,-0.211,-0.692***,-0.838***,P4


In [125]:
df_new.index.names=['portfolio_name', 'index']

In [115]:
df_new.groupby(['portfolio_name',df_new.index]).agg(lambda x:x)

excess_return        ff3        ff5       capm     carhrt  \
portfolio_name                                                                
P1             0       -1.751**  -3.087***  -3.061***  -2.659***  -3.093***   
               1        (-2.27)   (-15.34)   (-14.02)   (-10.73)   (-15.09)   
P10            18      5.295***   3.855***   3.969***   4.331***   3.865***   
               19        (5.77)    (15.55)    (15.48)    (12.18)    (15.61)   
P2             2         -0.456  -1.882***  -1.895***  -1.382***  -1.873***   
               3        (-0.54)   (-10.16)     (-9.3)    (-4.84)    (-10.3)   
P3             4          0.263  -1.085***  -1.038***   -0.635**  -1.063***   
               5         (0.34)    (-7.75)    (-6.71)    (-2.53)    (-7.57)   
P4             6          0.703  -0.707***  -0.635***     -0.211  -0.692***   
               7         (0.85)    (-4.54)     (-4.0)    (-0.68)    (-4.43)   
P5             8          1.314     -0.093     -0.097      0.395     -0.063   
               9         (1.57)    (-0.52)     (-0.6)     (1.24)    (-0.36)   
P6             10        1.467*      0.028      0.018     0.523*      0.045   
               11        (1.67)     (0.19)     (0.11)     (1.67)      (0.3)   
P7             12       2.111**   0.757***   0.727***   1.248***   0.773***   
               13        (2.51)     (4.88)     (4.93)     (3.69)     (5.01)   
P8             14       2.74***   1.251***   1.294***   1.786***   1.255***   
               15        (3.12)     (7.93)     (7.41)     (5.24)     (7.97)   
P9             16      3.771***   2.266***   2.268***   2.804***   2.244***   
               17        (4.29)    (12.72)    (13.07)     (7.54)    (12.28)   
h_l            20      7.047***   6.942***   7.031***   6.989***   6.959***   
               21       (13.31)    (17.08)    (16.24)    (17.07)    (17.03)   

                        lsy4  
portfolio_name                
P1             0   -3.137***  
               1    (-14.45)  
P10            18   3.867***  
               19     (14.7)  
P2             2   -2.047***  
               3    (-10.99)  
P3             4   -1.197***  
               5     (-7.61)  
P4             6   -0.838***  
               7     (-4.97)  
P5             8      -0.173  
               9     (-1.01)  
P6             10     -0.059  
               11    (-0.32)  
P7             12   0.645***  
               13     (4.43)  
P8             14    1.14***  
               15     (5.87)  
P9             16   2.092***  
               17    (11.51)  
h_l            20   7.004***  
               21    (16.49)

In [81]:
port_result_ew.set_index('portfolio_name',inplace=True)

In [76]:
pd.concat([port_result_ew[trans_i].apply(lambda x: ast.literal_eval(x)).explode() for trans_i in port_result_ew.columns],axis=1)

In [94]:
df_new =pd.concat([port_result_ew[trans_i].explode() for trans_i in port_result_ew.columns],axis=1).reset_index()

In [102]:
df_new.groupby(['portfolio_name',df_new.index]).agg(lambda x:x)

excess_return        ff3        ff5       capm     carhrt  \
portfolio_name                                                                
P1             0       -1.751**  -3.087***  -3.061***  -2.659***  -3.093***   
               1        (-2.27)   (-15.34)   (-14.02)   (-10.73)   (-15.09)   
P10            18      5.295***   3.855***   3.969***   4.331***   3.865***   
               19        (5.77)    (15.55)    (15.48)    (12.18)    (15.61)   
P2             2         -0.456  -1.882***  -1.895***  -1.382***  -1.873***   
               3        (-0.54)   (-10.16)     (-9.3)    (-4.84)    (-10.3)   
P3             4          0.263  -1.085***  -1.038***   -0.635**  -1.063***   
               5         (0.34)    (-7.75)    (-6.71)    (-2.53)    (-7.57)   
P4             6          0.703  -0.707***  -0.635***     -0.211  -0.692***   
               7         (0.85)    (-4.54)     (-4.0)    (-0.68)    (-4.43)   
P5             8          1.314     -0.093     -0.097      0.395     -0.063   
               9         (1.57)    (-0.52)     (-0.6)     (1.24)    (-0.36)   
P6             10        1.467*      0.028      0.018     0.523*      0.045   
               11        (1.67)     (0.19)     (0.11)     (1.67)      (0.3)   
P7             12       2.111**   0.757***   0.727***   1.248***   0.773***   
               13        (2.51)     (4.88)     (4.93)     (3.69)     (5.01)   
P8             14       2.74***   1.251***   1.294***   1.786***   1.255***   
               15        (3.12)     (7.93)     (7.41)     (5.24)     (7.97)   
P9             16      3.771***   2.266***   2.268***   2.804***   2.244***   
               17        (4.29)    (12.72)    (13.07)     (7.54)    (12.28)   
h_l            20      7.047***   6.942***   7.031***   6.989***   6.959***   
               21       (13.31)    (17.08)    (16.24)    (17.07)    (17.03)   

                        lsy4  
portfolio_name                
P1             0   -3.137***  
               1    (-14.45)  
P10            18   3.867***  
               19     (14.7)  
P2             2   -2.047***  
               3    (-10.99)  
P3             4   -1.197***  
               5     (-7.61)  
P4             6   -0.838***  
               7     (-4.97)  
P5             8      -0.173  
               9     (-1.01)  
P6             10     -0.059  
               11    (-0.32)  
P7             12   0.645***  
               13     (4.43)  
P8             14    1.14***  
               15     (5.87)  
P9             16   2.092***  
               17    (11.51)  
h_l            20   7.004***  
               21    (16.49)